In [1]:
import math
import numpy as np
import sympy as sp
import scipy
import IPython.display as dp

**Вариант 18.**<br>
$f(x) = x^2 \sin(x)$<br>
$a = -1$, $b = 1$<br>

$x^{(1)} = -0.89, x^{(2)} = -0.42, x^{(3)} = -0.26$

In [32]:
x = sp.symbols('x')
f = x**2 * sp.sin(x)
a = -1
b = 1
X = [-0.89, -0.42, -0.26] # Контрольные точки x^(i)
Y = [f.subs(x, x_l) for x_l in X]

**Задача 1.**<br>

Для функции $f(x), x \in [a, b]$, построить интерполяционный сплайн первой степени по следующим узлам:
1. $x_k = a + k \frac{b-a}{4}$, $k = \overline{0, 4}$
2. $x_k = \frac{b -a}{2} \cos{\frac{(2k - 1)\pi}{10}} + \frac{b+a}{2}$, $k = \overline{1,5}$

In [3]:
# Первый случай
X1 = [(a + k * (b - a)/4) for k in range(5)]
Y1 = [f.subs(x, x_k) for x_k in X1]
print(f"x_k = {X1}")
print(f"y_k = {Y1}")

x_k = [-1.0, -0.5, 0.0, 0.5, 1.0]
y_k = [-0.841470984807897, -0.119856384651051, 0, 0.119856384651051, 0.841470984807897]


In [4]:
# Второой случай
X2 = [((b-a)/2 * np.cos((2*k-1)*np.pi / 10) + (b+a)/2) for k in range(1, 6)]   # x_k
Y2 = [f.subs(x, x_k) for x_k in X2]   # y_k
print(f"x_k = {X2}")
print(f"y_k = {Y2}")

x_k = [0.9510565162951535, 0.5877852522924731, 6.123233995736766e-17, -0.587785252292473, -0.9510565162951535]
y_k = [0.736296697723516, 0.191581718715425, 2.29584502165847e-49, -0.191581718715425, -0.736296697723516]


In [8]:
def spline_interpolate(X, Y, x):
    S = [((Y[k+1] - Y[k]) * (x - X[k+1]) / (X[k+1] - X[k]) + Y[k+1]) for k in range(len(X) - 1)]
    return np.array(S)[:, None]

In [34]:
display_sol(spline_interpolate(X1, Y1, x))

$\begin{cases}1.44322920031369 x + 0.601758215505795\\0.239712769302102 x\\0.239712769302102 x\\1.44322920031369 x - 0.601758215505795\\\end{cases}$

In [35]:
display_sol(spline_interpolate(X2, Y2, x))

$\begin{cases}1.49947169783314 x - 0.689785631500853\\0.325938287781499 x - 1.99579640425591 \cdot 10^{-17}\\0.325938287781499 x - 5.55111512312578 \cdot 10^{-17}\\1.49947169783314 x + 0.689785631500853\\\end{cases}$

---
**Задача 2.1**<br>

Для функции $f(x), x \in [a, b]$ построить естественный сплайн третей степени, исползуя определение

In [39]:
def cubspline_interpolate(X, Y, x):
    M = __get_matrixes(X, Y)
    solution = np.linalg.solve(M[0], M[1])
    solution = np.reshape(solution, (len(X)-1, 4))

    return np.dot(solution, np.array([1, x, x**2, x**3]).T)[:, None]

def __get_matrixes(X, Y):
    nsegs = len(X) - 1 # Number of segments
    n = 4*(nsegs)
    cub   = lambda x, y: np.array([[1, x, x**2, x**3],
                                   [1, y, y**2, y**3]])
    dcub  = lambda x: np.array([0, 1, 2*x, 3*x**2, 0, -1, -2*x, -3*x**2])
    ddcub = lambda x: np.array([0, 0, 2, 6*x])

    # Добавление условий интерполяции
    int_rules = np.zeros((nsegs, 2, n))
    sample = np.zeros((2, n-4))
    for k in range(nsegs):
        int_rules[k] = np.insert(sample, 4 * k, cub(X[k], X[k+1]).T, axis=1)
    int_rules = np.reshape(int_rules, (int_rules.size))

    # Условия гладкой склейки на внутренних узловых точках
    sm_rules = np.zeros((nsegs-1, n))  # Smooth gluing rules
    sample = np.zeros((n-8, ))
    for k in range(nsegs-1):
        sm_rules[k] = np.insert(sample, 4*k, dcub(X[k+1]).T)

    # Условия дважды гладкой склейки на внутренних узловых точках
    dsm_rules = np.zeros((nsegs-1, n))
    for k in range(nsegs-1):
        dsm_rules[k] = np.insert(sample, 4*k, np.append(ddcub(X[k+1]), ddcub(-X[k+1])).T)
    
    # Добавление остаточных условий
    res = np.array(ddcub(X[0]))
    res.resize(2 * n - 4)
    res = np.append(res, ddcub(X[-1]))

    # Объединение всех условий в одну матрицу коэф СЛАУ
    int_rules = np.append(int_rules, [sm_rules, dsm_rules])
    res = np.append(int_rules, res)
    res.resize((n, n))

    # Добавление вектора-столбца свободных членов СЛАУ
    resY = np.zeros(n)
    for k in range(nsegs):
        resY[2*k] = Y[k]
        resY[2*k+1] = Y[k+1]
        
    return [res, resY]


def display_sol(solution):
    string = "\\begin{cases}"
    for sol in solution:
        string += str(sp.latex(sol[0]))
        string += "\\\\"
    string += "\\end{cases}"
    dp.display(dp.Markdown(rf"${string}$"))

In [40]:
solution1 = cubspline_interpolate(X1, Y1, x)
display_sol(solution1)

$\begin{cases}0.802344287341059 x^{3} + 2.40703286202318 x^{2} + 3.6496759905016 x + 1.20351643101159\\0.80234428734106 x^{3} - 2.40703286202318 x^{2} - 1.16438973354475 x - 1.11022302462516 \cdot 10^{-16}\\0.802344287341061 x^{3} + 2.40703286202318 x^{2} - 1.16438973354475 x\\0.80234428734106 x^{3} - 2.40703286202318 x^{2} + 3.64967599050161 x - 1.20351643101159\\\end{cases}$

In [41]:
solution2 = cubspline_interpolate(X2, Y2, x)
display_sol(solution2)

$\begin{cases}0.557735998005517 x^{3} - 1.59131536582658 x^{2} + 2.93930035099574 x - 1.09957187437619\\0.557735998005521 x^{3} + 1.59131536582659 x^{2} - 0.802106464121344 x + 5.55111512312578 \cdot 10^{-17}\\0.557735998005522 x^{3} - 1.59131536582659 x^{2} - 0.802106464121344 x + 1.66533453693773 \cdot 10^{-16}\\0.55773599800552 x^{3} + 1.59131536582659 x^{2} + 2.93930035099575 x + 1.09957187437619\\\end{cases}$

**Задача 2.2**

Для функции $f(x), x \in [a, b]$ построить естественный сплайн третей степени через метод моментов